<a href="https://colab.research.google.com/github/BenUCL/Reef-acoustics-and-AI/blob/main/Code/Random_forests_compound_index_Poly_habitat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Random forest classifiers**

This script splits the data into the same train/val/test sets as used for the pretrained CNN and trained CNN. It then trains on the training data and generates the accuracy for the validation data 50 times. The best performing model of the 50 repeats is used to inference on the test data and this accuracy is reported. This proces is repeated 100 times to generate an accuracyfor all train/val/test split combinations. A confusion matrix of all these is then generated.

For polynesia site level classfication I take 1/4 days from each of the 8 sites at random as the val and test set. Every minute from this day is put in the test and every other in the val. So each of the test and val have recordings from all 8 sites. If i put 4 enitre sites in the val, and 4 in the test, then it would train to fit the val well, and do crap on the test.

In [ ]:
# Connect Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# imports, probably some redundant packages
from __future__ import division

import sklearn
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd
from sklearn.model_selection import train_test_split
import statistics

from sklearn.metrics import confusion_matrix

import collections
from sklearn.metrics import ConfusionMatrixDisplay

import seaborn as sn
import matplotlib.pyplot as plt

In [ ]:
# Upload the csv of features and copy the path
path = r'/content/drive/MyDrive/Reef soundscapes with AI/Results/full_dataset_features/compound_index_poly.csv'
num_classes = 2
labels = ['Photic', 'Mesophotic']

In [ ]:
data = pd.read_csv (path)
#Use the above functions to get a list of unique deployment ID's (approx 30 for healthy, and again for degraded)
df = data.reset_index() 
df = df.iloc[: , 2:] #remove additional index columns that got added at start
df

In [ ]:
def get_class(filename):
    #find part of the name that corresponds to the deployment
     #adapted the get_identifier function above to only get class (e.g healthy)
    site = filename[0:5]
    return site
    
df_withclasses = df

# add a column to the DF that contains class
new_list = []
for i in range(len(df_withclasses)):
    new_list.append(get_class(df_withclasses['minute'][i]))

df_withclasses.insert(1, 'Class', new_list)

indices_df = df_withclasses 

# Rename the remaiming sites to mesophotic or photic fish
indices_df['Class'] = indices_df['Class'].replace(['SiteA'],'Photic')
indices_df['Class'] = indices_df['Class'].replace(['SiteB'],'Photic')
indices_df['Class'] = indices_df['Class'].replace(['SiteC'],'Photic')
indices_df['Class'] = indices_df['Class'].replace(['SiteD'],'Photic')

indices_df['Class'] = indices_df['Class'].replace(['SiteW'],'Mesophotic')
indices_df['Class'] = indices_df['Class'].replace(['SiteX'],'Mesophotic')
indices_df['Class'] = indices_df['Class'].replace(['SiteY'],'Mesophotic')
indices_df['Class'] = indices_df['Class'].replace(['SiteZ'],'Mesophotic')

indices_df.reset_index(drop=True, inplace=True)
indices_df.head()

In [ ]:
def similar(x,y):
    si = 0
    for a,b in zip(x, y):
        if a == b:
            si += 1
    return (si/len(x)) * 100

def get_class(filename):
    #find part of the name that corresponds to the deployment
     #adapted the get_identifier function above to only get class (e.g healthy)
    site = filename[0:5]
    return site
    
df_withclasses = df

# add a column to the DF that contains class
new_list = []
for i in range(len(df_withclasses)):
    new_list.append(get_class(indices_df['minute'][i]))

indices_df.insert(1, 'Site', new_list)

In [ ]:
all_saved_test_accs = []
whole_site_saved_test_accs = []
ConfusionMatrix = np.zeros((num_classes, num_classes), dtype=float)

for i in range(4):  #4 sites in the photicfish class
  photic = ['SiteA', 'SiteB', 'SiteC', 'SiteD'] #the 4 photic fish sites
  remove_photic = photic[i] #select 1 to be the val site
  train_photic = photic 
  train_photic.remove(remove_photic) #remove the val site to get the train photicfish sites
  for k in range(4): #4 sites in the mesophoticfish class
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []

    mesophotic = ['SiteW', 'SiteX', 'SiteY', 'SiteZ'] #the 4 mesophotic fish sites
    remove_mesophotic = mesophotic[k] #select 1 to be the test site
    train_mesophotic = mesophotic 
    train_mesophotic.remove(remove_mesophotic) #remove the test site to get the train photicfish sites
    
    # Make the train df's, starting with photic fish then append mesophotic fish
    train_df = indices_df.loc[indices_df['Site'].isin(train_photic)]
    train_df = train_df.append(indices_df.loc[indices_df['Site'].isin(train_mesophotic)])

    # create val df that is 50:50 photic and mesophotic fish
    photicfish_df = indices_df.loc[indices_df['Site'] == remove_photic]
    mesophoticfish_df = indices_df.loc[indices_df['Site'] == remove_mesophotic]
    val_df = photicfish_df.iloc[::2,:] # take every other photicfish minute and create a new df
    val_df = val_df.append(mesophoticfish_df.iloc[::2,:])
    # create test df's that is the ramaining 50:50 photic and mesophotic fish
    test_df = photicfish_df.iloc[1::2,:] # take every other photicfish minute and create a new df
    test_df = test_df.append(mesophoticfish_df.iloc[1::2,:])

    #shuffle the df's so the RF's aren't first fed all photic then fed all mesophotic fish after
    np.random.seed(123) 
    train_df = train_df.sample(frac=1)
    val_df = val_df.sample(frac=1)
    test_df = test_df.sample(frac=1)
    np.random.seed() #now lift the seed so that randomisation can be used again in the rest of the script

    # Make new df's of the feats and labels to feed random forests
    train_feats = train_df.iloc[:, 3:].to_numpy()
    train_labels = train_df.iloc[:, 2].to_numpy()

    val_feats = val_df.iloc[:, 3:].to_numpy()
    val_labels = val_df.iloc[:, 2].to_numpy()

    test_feats = test_df.iloc[:, 3:].to_numpy()
    test_labels = test_df.iloc[:, 2].to_numpy()

    
    val_accuracy_score = 0
    for k in range(50):              # Picked 50 as 50 epochs used in NN
      #Training on train data
      model = RandomForestClassifier(n_jobs = -1, random_state=k)
      model.fit(train_feats, train_labels)

      #Inferencing on validation data
      new_val_acc = model.score(val_feats, val_labels)

      #If val acc improved, inference on test data
      if new_val_acc >  val_accuracy_score:
        #save the new mesophotic score for val
        val_accuracy_score = new_val_acc 

        #get 1min acc for test data
        test_acc = model.score(test_feats, test_labels) 
        
        #get the acc when using the most common prediction across all minutes in a day for each site
        test_predictions = model.predict(test_feats)

        # get confusion matrix values
        best_ConfusionMatrix = confusion_matrix(test_labels, test_predictions, labels = labels) ###

        #acc = 0 
        #for i in range(num_classes):
          # max = np.amax(best_ConfusionMatrix[i])
          #if max == best_ConfusionMatrix[i,i]:
            # acc += 1
            #whole_day_acc = acc/num_classes
        #print(similar(test_labels, test_predictions))
        if  similar(test_labels, test_predictions) >= 50:
          whole_site_acc = 1
        else:
          whole_site_acc = 0
        
      
    #Save best results
    ConfusionMatrix = np.add(ConfusionMatrix, best_ConfusionMatrix) ###
    thisCV_saved_test_accs.append(test_acc)
    all_saved_test_accs.append(test_acc)
    thisCV_saved_test_day_accs.append(whole_site_acc)
    whole_site_saved_test_accs.append(whole_site_acc)

    print('Accuracies for 1min files from this site:')
    print(thisCV_saved_test_accs)
    print('Accuracies for the whole site [0 or 1]:')
    print(thisCV_saved_test_day_accs)
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []
    print()


pt1_saved_test_accs = all_saved_test_accs
pt1_whole_site_saved_test_accs = whole_site_saved_test_accs


# this will run all the lowfish sites as val, then repeat the whole thing below but swap the highfish to be in the top loop and low fish in bottom loop
    
    
print('Part 1 Completed RFs:')
print(len(all_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(all_saved_test_accs)
stdev = np.std(all_saved_test_accs)

print('1min saved_test_accs: ')
print(all_saved_test_accs)
result = 'Mean accuracy for 1min files with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result)     


print('Part 1 Completed RFs:')
print(len(whole_site_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(whole_site_saved_test_accs)
stdev = np.std(whole_site_saved_test_accs)

print('Whole site saved test_accs: ')
print(whole_site_saved_test_accs)
result = 'Mean accuracy for whole sites with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result)


all_saved =  mean_accuracy
stdev_pt1 = stdev




all_saved_test_accs = []
whole_site_saved_test_accs = []
#ConfusionMatrix = np.zeros((num_classes, num_classes), dtype=float)

for i in range(4):  #4 sites in the photicfish class
  mesophotic =  ['SiteA', 'SiteB', 'SiteC', 'SiteD'] #the 4 mesophotic fish sites
  remove_mesophotic = mesophotic[i] #select 1 to be the val site
  train_mesophotic = mesophotic 
  train_mesophotic.remove(remove_mesophotic) #remove the val site to get the train photicfish sites
  #print(train_photic)
  for k in range(4): #4 sites in the mesophoticfish class
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []

    photic = ['SiteW', 'SiteX', 'SiteY', 'SiteZ'] #the 4 photic fish sites
    remove_photic = photic[k] #select 1 to be the test site
    train_photic = photic 
    train_photic.remove(remove_photic) #remove the test site to get the train photicfish sites
    
    # Make the train df's, starting with photic fish then append mesophotic fish
    train_df = indices_df.loc[indices_df['Site'].isin(train_photic)]
    train_df = train_df.append(indices_df.loc[indices_df['Site'].isin(train_mesophotic)])

    # create val df that is 50:50 photic and mesophotic fish
    photicfish_df = indices_df.loc[indices_df['Site'] == remove_photic]
    mesophoticfish_df = indices_df.loc[indices_df['Site'] == remove_mesophotic]
    val_df = photicfish_df.iloc[::2,:] # take every other photicfish minute and create a new df
    val_df = val_df.append(mesophoticfish_df.iloc[::2,:])
    # create test df's that is the ramaining 50:50 photic and mesophotic fish
    test_df = photicfish_df.iloc[1::2,:] # take every other photicfish minute and create a new df
    test_df = test_df.append(mesophoticfish_df.iloc[1::2,:])

    #shuffle the df's so the RF's aren't first fed all photic then fed all mesophotic fish after
    np.random.seed(123) 
    train_df = train_df.sample(frac=1)
    val_df = val_df.sample(frac=1)
    test_df = test_df.sample(frac=1)
    np.random.seed() #now lift the seed so that randomisation can be used again in the rest of the script

    # Make new df's of the feats and labels to feed random forests
    train_feats = train_df.iloc[:, 3:].to_numpy()
    train_labels = train_df.iloc[:, 2].to_numpy()

    val_feats = val_df.iloc[:, 3:].to_numpy()
    val_labels = val_df.iloc[:, 2].to_numpy()

    test_feats = test_df.iloc[:, 3:].to_numpy()
    test_labels = test_df.iloc[:, 2].to_numpy()

    
    val_accuracy_score = 0
    for k in range(50):              # Picked 50 as 50 epochs used in NN
      #Training on train data
      model = RandomForestClassifier(n_jobs = -1, random_state=k)
      model.fit(train_feats, train_labels)

      #Inferencing on validation data
      new_val_acc = model.score(val_feats, val_labels)

      #If val acc improved, inference on test data
      if new_val_acc >  val_accuracy_score:
        #save the new mesophotic score for val
        val_accuracy_score = new_val_acc 

        #get 1min acc for test data
        test_acc = model.score(test_feats, test_labels) 
        
        #get the acc when using the most common prediction across all minutes in a day for each site
        test_predictions = model.predict(test_feats)

        # get confusion matrix values
        best_ConfusionMatrix = confusion_matrix(test_labels, test_predictions, labels = labels) ###

        if  similar(test_labels, test_predictions) >= 50:
          whole_site_acc = 1
        else:
          whole_site_acc = 0
        
      
    #Save best results
    ConfusionMatrix = np.add(ConfusionMatrix, best_ConfusionMatrix) ###
    thisCV_saved_test_accs.append(test_acc)
    all_saved_test_accs.append(test_acc)
    thisCV_saved_test_day_accs.append(whole_site_acc)
    whole_site_saved_test_accs.append(whole_site_acc)

    print('Accuracies for 1min files from this site:')
    print(thisCV_saved_test_accs)
    print('Accuracies for the whole site [0 or 1]:')
    print(thisCV_saved_test_day_accs)
    thisCV_saved_test_accs = []
    thisCV_saved_test_day_accs = []
    print()



both_saved_test_accs = pt1_saved_test_accs + all_saved_test_accs 
both_whole_site_saved_test_accs = pt1_whole_site_saved_test_accs + whole_site_saved_test_accs
    
print('Part 2 Completed RFs:')
print(len(both_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(both_saved_test_accs)
stdev = np.std(both_saved_test_accs)

print('1min saved_test_accs: ')
print(both_saved_test_accs)
result = 'Mean accuracy for 1min files with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result) 




print('Part 2 Completed RFs:')
print(len(both_whole_site_saved_test_accs))

def Average(lst):
    return sum(lst) / len(lst)
mean_accuracy = Average(both_whole_site_saved_test_accs)
stdev = np.std(both_whole_site_saved_test_accs)

print('Whole site saved test_accs: ')
print(both_whole_site_saved_test_accs)
result = 'Mean accuracy for whole sites with standard deviation = {} (±{})'.format(str(mean_accuracy), str(stdev))
print(result) 

print()
print(repr(ConfusionMatrix))



executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))


In [ ]:
#HPConfusionMatrix = [[1.3010e+04, 8.4000e+01, 0.0000e+00, 4.0000e+00, 0.0000e+00,
      #   2.3000e+01, 0.0000e+00, 2.0000e+00],
      #  [2.0500e+02, 1.4088e+04, 0.0000e+00, 1.3000e+01, 0.0000e+00,
      #   0.0000e+00, 0.0000e+00, 0.0000e+00],
      #  [0.0000e+00, 0.0000e+00, 1.4395e+04, 0.0000e+00, 2.0000e+00,
      #   1.5000e+01, 2.0000e+01, 2.0000e+00],
      #  [5.3000e+01, 7.0000e+00, 1.2000e+01, 1.4226e+04, 1.5000e+01,
      #   1.9000e+01, 0.0000e+00, 4.9000e+01],
      #  [0.0000e+00, 0.0000e+00, 4.0000e+00, 0.0000e+00, 1.3063e+04,
      #   1.4100e+02, 8.0000e+00, 1.4000e+01],
      #  [0.0000e+00, 0.0000e+00, 1.3000e+01, 0.0000e+00, 7.0000e+00,
      #   1.4187e+04, 2.0000e+00, 5.3000e+01],
      #  [0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 1.3000e+01,
      #   1.6000e+01, 1.4200e+04, 1.5900e+02],
      #  [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.8000e+01,
      #   4.2000e+01, 0.0000e+00, 1.4340e+04]]

array = ConfusionMatrix
df_cm = pd.DataFrame(array, index = [i for i in "ABCDWXYZ"],
                  columns = [i for i in "ABCDWXYZ"])
plt.figure(figsize = (15,12))
cmap = sn.cm.rocket_r
ax = sn.heatmap(df_cm, annot=True, annot_kws={"fontsize":20}, fmt='g', cmap = cmap)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize = 20)
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize = 20)
cbar = ax.collections[0].colorbar
cbar.ax.tick_params(labelsize=20)